## 여러 페이지의 데이터를 항목별로 수집후 다양한 형식으로 저장하기

## 연습 문제로 실력 굳히기

1. 사용자에게 RISS 사이트의 모든 카테고리를 다 보여준 후 사용자에게 '국내학술논문' 카테고리를 선택했다고 가정하여 수집할 장르를 2번으로 입력한 후 저장할 파일의 이름을 입력 받은 후 '국내학술논문' 카테고리의 다양한 정보들을 수집하여 CSV, xls 형식으로 저장하도록 코드를 작성하세요. 수집할 항목은 아래의 예시를 참고하세요.

In [21]:
# riss.kr 에서 특정 키워드로 논문 / 학술 자료 검색하기

#Step 1. 필요한 모듈을 로딩합니다
from selenium import webdriver
import time 

#Step 2. 사용자에게 검색 관련 정보들을 입력 받습니다.
print("=" *100)
print(" 이 크롤러는 RISS 사이트의 논문 및 학술자료 수집용 웹크롤러입니다.")
print("=" *100)

query_txt = input('1.수집할 자료의 키워드는 무엇입니까?(여러개일 경우 , 로 구분하여 입력): ')

# 장르 선택 받기
print('2. 위 키워드로 아래의 장르 중 어떤 장르의 정보를 수집할까요?')
print('1.학위논문\t 2.국내학술논문\t 3.해외학술논문\t 4.학술지')
print('5.단행본\t 6.공개강의\t 7.연구보고서')
genre = input('위 장르 중 수집할 장르의 번호를 입력하세요 : ')

#Step 3. 수집된 데이터를 저장할 파일 이름 입력받기 
ft_name = input('2.결과를 저장할 txt형식의 파일명을 쓰세요(예: c:\\temp\\riss.txt): ')
fc_name = input('3.결과를 저장할 csv형식의 파일명을 쓰세요(예: c:\\temp\\riss.csv): ')
fx_name = input('4.결과를 저장할 xls형식의 파일명을 쓰세요(예: c:\\temp\\riss.xls): ')

#Step 4. 크롬 드라이버 설정 및 웹 페이지 열기
chrome_path = "C:/Users/aa/Downloads/chromedriver.exe"
driver = webdriver.Chrome(chrome_path)

url = 'http://www.riss.kr/'
driver.get(url)
time.sleep(5)

#Step 5. 자동으로 검색어 입력 후 조회하기
element = driver.find_element_by_id("query")
driver.find_element_by_id("query").click( )
element.send_keys(query_txt)
element.send_keys("\n")
time.sleep(5)

#Step 6.학위 논문 선택하기
if genre == '1' :
    print('학위논문')
    driver.find_element_by_link_text('학위논문').click()
elif genre == '2' :
    print('국내학술논문')
    driver.find_element_by_link_text('국내학술논문').click()
elif genre == '3' :
    print('해외학술논문')
    driver.find_element_by_link_text('해외학술논문').click()
elif genre == '4' :
    print('학술지')
    driver.find_element_by_link_text('학술지').click()
elif genre == '5' :
    print('단행본')
    driver.find_element_by_link_text('단행본').click()
elif genre == '6' :
    print('공개강의')
    driver.find_element_by_link_text('공개강의').click()
elif genre == '7' :
    print('연구보고서')
    driver.find_element_by_link_text('연구보고서').click()

time.sleep(5)

#Step 7.Beautiful Soup 로 본문 내용만 추출하기
from bs4 import BeautifulSoup
html_1 = driver.page_source
soup_1 = BeautifulSoup(html_1, 'html.parser')

content_1 = soup_1.find('div','srchResultListW').find_all('li')
for i in content_1 :
    print(i.get_text().replace("\n",""))

#Step 8. 총 검색 건수를 보여주고 수집할 건수 입력받기
import math
total_cnt = soup_1.find('div','searchBox').find('span','num').get_text()
print('검색하신 키워드 %s (으)로 총 %s 건의 학위논문이 검색되었습니다' %(query_txt,total_cnt))
collect_cnt = int(input('이 중에서 몇 건을 수집하시겠습니까?: '))
collect_page_cnt = math.ceil(collect_cnt / 10)
print('%s 건의 데이터를 수집하기 위해 %s 페이지의 게시물을 조회합니다.' %(collect_cnt,collect_page_cnt))

#Step 9. 각 항목별로 데이터를 추출하여 리스트에 저장하기
no2 = [ ]        #번호 저장
title2 = [ ]     #논문제목 저장
writer2 = [ ]    #논문저자 저장
org2 = [ ]       #소속기관 저장
year2 = []       #발표년도 저장
paper_zip2 = []  #논문집/발표집 저장
paper_url2 = []  #논문 url 주소 저장
no = 1

# 다음 페이지 번호 만들기
page_no=[ ]

for i in range(10,collect_cnt) :
    if i % 10 == 0 :
            page_no.append(i + 1)

for a in range(1, collect_page_cnt + 1) :
    
    html_2 = driver.page_source
    soup_2 = BeautifulSoup(html_2, 'html.parser')

    content_2 = soup_2.find('div','srchResultListW').find_all('li')
    
    for b in content_2 :    
        #1. 논문제목 있을 경우만
        try :
            title = b.find('div','cont').find('p','title').get_text()
        except :
            continue
        else :
            f = open(ft_name, 'a' , encoding="UTF-8")
            print('1.번호:',no)
            no2.append(no)
            f.write('\n'+'1.번호:' + str(no))

            print('2.제목:',title)
            title2.append(title)
            f.write('\n' + '2.제목:' + title)
            
            writer = b.find('span','writer').get_text()
            print('3.작성자:',writer)
            writer2.append(writer)
            f.write('\n' + '3.작성자:' + writer)

            org = b.find('span','assigned').get_text()
            print('4.소속기관:' , org)
            org2.append(org)
            f.write('\n' + '4.소속기관:' + org + '\n')
            
            year = b.find('p','etc').find_all('span')[2].get_text()
            print('5.발표년도:', year)
            year2.append(year)
            f.write('\n' + '5.발표년도:' + year + '\n')
            
            paper_zip = b.find('p','etc').find_all('span')[3].get_text()
            print('6.논문집/자료집:', paper_zip)
            paper_zip2.append(paper_zip)
            f.write('\n' + '6.논문집/자료집:' + paper_zip + '\n')
            
            paper_url = url + b.find('div','cont').find('p','title').find('a')['href']
            print('7.논문 URL 주소:', paper_url)
            paper_url2.append(paper_url)
            f.write('\n' + '7.논문 URL 주소:' + paper_url + '\n')
            
            f.close( )
            
            no += 1
            print("\n")
                        
            if no > collect_cnt :
                break

            time.sleep(1)        # 페이지 변경 전 1초 대기 

    c = math.floor(a/10)-1 
    a += 1

    if a == page_no[c]:
        driver.find_element_by_link_text('다음 페이지로').click()
    else :
        driver.find_element_by_link_text('%s' %a).click() # 다음 페이지번호 클릭
        
print("요청하신 작업이 모두 완료되었습니다")

# Step 10. 수집된 데이터를 xls와 csv 형태로 저장하기
import pandas as pd 

df = pd.DataFrame()
df['번호']=no2
df['제목']=pd.Series(title2)
df['저자']=pd.Series(writer2)
df['소속(발행)기관']=pd.Series(org2)
df['날짜']=pd.Series(year2)
df['학위(논문일경우)']=pd.Series(paper_zip2)
df['자료URL주소']=pd.Series(paper_url2)

# xls 형태로 저장하기
df.to_excel(fx_name,index=False, encoding="utf-8")

# csv 형태로 저장하기
df.to_csv(fc_name,index=False, encoding="utf-8-sig")

print('요청하신 데이터 수집 작업이 정상적으로 완료되었습니다')


 이 크롤러는 RISS 사이트의 논문 및 학술자료 수집용 웹크롤러입니다.
1.수집할 자료의 키워드는 무엇입니까?(여러개일 경우 , 로 구분하여 입력): 해양자원
2. 위 키워드로 아래의 장르 중 어떤 장르의 정보를 수집할까요?
1.학위논문	 2.국내학술논문	 3.해외학술논문	 4.학술지
5.단행본	 6.공개강의	 7.연구보고서
위 장르 중 수집할 장르의 번호를 입력하세요 : 2
2.결과를 저장할 txt형식의 파일명을 쓰세요(예: c:\temp\riss.txt): e:\\BigAI\\riss.txt
3.결과를 저장할 csv형식의 파일명을 쓰세요(예: c:\temp\riss.csv): e:\\BigAI\\riss.csv
4.결과를 저장할 xls형식의 파일명을 쓰세요(예: c:\temp\riss.xls): e:\\BigAI\\riss.xls
국내학술논문
1발표논문 / 해양생물자원으로서 해조류 : 생물활성물질의 정제와 분자적 응용홍용기(Yong Ki Hong)한국조류학회2000국제심포지움 일정 및 발표논문집 - 21세기, 해양환경과 해양생물자원의 전망Vol.- No.-해조생물공학은 유용해조류 및 그들의 유용물질들을 경제적으로 생산할 수 있도록 개발하는 학문 분야이다. 본 논문에서는 이들 해조생물공학에 대한 실험방법적 접근을 강조하여 설명하고자 한다. 우선 해조류의 활력측정은 조직 자체의 효소활성 즉 무색의 2,3,5-triphenyltetrazolium chloride를 붉은색의 triphenylformazan으로 환원시키는 정도를 측정함으로서 정량적으로 간단하게 활력을 평가하는 방법을 확립하였다. 그리고 callus와 엽체 재생 등을 위한 무균 고체 조직배양에 가장 적합한 gelling agent들을 비교한바 high gel strength agar가 톳의 callus 형성 및 엽체 재생에 지지체로 가장 효율적이였다. 유용 생리활성물질의 탐색작업에서는 보라색우무 추출물에서 Taq DNA polymerase에 대한 저해작용을 보였으며, 청각, 구멍갈파래, 잎파래, 미

이 중에서 몇 건을 수집하시겠습니까?: 30
30 건의 데이터를 수집하기 위해 3 페이지의 게시물을 조회합니다.
1.번호: 1
2.제목: 발표논문 / 해양생물자원으로서 해조류 : 생물활성물질의 정제와 분자적 응용
3.작성자: 홍용기(Yong Ki Hong)
4.소속기관: 한국조류학회
5.발표년도: 2000
6.논문집/자료집: 국제심포지움 일정 및 발표논문집 - 21세기, 해양환경과 해양생물자원의 전망
7.논문 URL 주소: /search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=508c47ca1906d5ecffe0bdc3ef48d419


2
1.번호: 2
2.제목: 수거된 해양폐기물 자원화 기술 개발(Ⅰ)
3.작성자: 길상인(Sang-In Keel), 윤진한(Jin-Han Yun), 최연석(Yeon-Seok Choi), 강창구(Chang-Gu Kang), 유정석(Jeong-Seok Yu)
4.소속기관: 한국해양환경·에너지학회
5.발표년도: 2002
6.논문집/자료집: 한국해양환경·에너지학회지
7.논문 URL 주소: /search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=09ec1e3541fcc118ffe0bdc3ef48d419


3
1.번호: 3
2.제목: 해양경찰 미래 발전을 위한 인적 자원 확보 방안연구
3.작성자: 윤병두
4.소속기관: 한국해양경찰학회
5.발표년도: 2021
6.논문집/자료집: 한국해양경찰학회보
7.논문 URL 주소: /search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=fcc6dea7df1a2c0fd18150b21a227875


4
1.번호: 4
2.제목: 침적 해양폐기물 수거사업과 자원 및 해양환경 조사사업의 연계방안에 대한 고찰
3.작성자: 김정협(Jeong-Hyop Kim), 장철호(Cheol-Ho Jang), 김광태(Gwang 

2. 네이버 (www.naver.com)에서 아래 예시와 같이 사용자에게 검색어와 저장할 파일들의 경로를 입력 받은 후 해당 키워드로 검색하여 나오는 결과에서 블로그만 선택한 후 30건의 데이터를 아래 예시와 같이 xls, csv 형식으로 저장하세요.

(편의상 검색어는 '서수진 빅데이터'로 해보세요)

# 상세 정보 수집 후 다양한 형식의 파일로 저장하기

논문으로 들어가서 상세 내역을 뽑아오고 돌아와서

다음 논문으로 들어가서 상세 내역을 뽑아오고 반복

1. 사용자에게 RISS 사이트의 모든 카테고리를 다 보여준 후 사용자에게 '국네학술논문' 카테고리를 선택했다고 가정하여 수집할 장르를 2번으로 입력 후 저장할 폴더의 이름을 입력 받은 후 '국내학술논문' 카테고리의 다양한 정보들을 수집하여 csv, xls 형식으로 저장하도록 코드를 작성하세요. 수집할 항목은 아래의 예시를 참고하세요.

In [39]:
# riss.kr 에서 특정 키워드로 논문 / 학술 자료 검색하기

#Step 1. 필요한 모듈을 로딩합니다
from selenium import webdriver
import time 

#Step 2. 사용자에게 검색 관련 정보들을 입력 받습니다.
print("=" *100)
print(" 이 크롤러는 RISS 사이트의 논문 및 학술자료 수집용 웹크롤러입니다.")
print("=" *100)

query_txt = input('1.수집할 자료의 키워드는 무엇입니까?(여러개일 경우 , 로 구분하여 입력): ')

# 장르 선택 받기
print('2. 위 키워드로 아래의 장르 중 어떤 장르의 정보를 수집할까요?')
print('1.학위논문\t 2.국내학술논문\t 3.해외학술논문\t 4.학술지')
print('5.단행본\t 6.공개강의\t 7.연구보고서')
genre = input('위 장르 중 수집할 장르의 번호를 입력하세요 : ')

#Step 3. 수집된 데이터를 저장할 파일 이름 입력받기 
dir_path = input("3.파일을 저장할 폴더명만 쓰세요(기본값 : e:\\BigAI\\) : ")

if dir_path == '' :
    dir_path = 'e:\\BigAI\\'

#Step 4. 크롬 드라이버 설정 및 웹 페이지 열기
chrome_path = "C:/Users/aa/Downloads/chromedriver.exe"
driver = webdriver.Chrome(chrome_path)

url = 'http://www.riss.kr/'
driver.get(url)
time.sleep(5)

#Step 5. 자동으로 검색어 입력 후 조회하기
element = driver.find_element_by_id("query")
driver.find_element_by_id("query").click( )
element.send_keys(query_txt)
element.send_keys("\n")
time.sleep(5)

#Step 6.학위 논문 선택하기
if genre == '1' :
    print('학위논문')
    driver.find_element_by_link_text('학위논문').click()
elif genre == '2' :
    print('국내학술논문')
    driver.find_element_by_link_text('국내학술논문').click()
elif genre == '3' :
    print('해외학술논문')
    driver.find_element_by_link_text('해외학술논문').click()
elif genre == '4' :
    print('학술지')
    driver.find_element_by_link_text('학술지').click()
elif genre == '5' :
    print('단행본')
    driver.find_element_by_link_text('단행본').click()
elif genre == '6' :
    print('공개강의')
    driver.find_element_by_link_text('공개강의').click()
elif genre == '7' :
    print('연구보고서')
    driver.find_element_by_link_text('연구보고서').click()

time.sleep(5)

#Step 7.Beautiful Soup 로 본문 내용만 추출하기
from bs4 import BeautifulSoup
html_1 = driver.page_source
soup_1 = BeautifulSoup(html_1, 'html.parser')

content_1 = soup_1.find('div','srchResultListW').find_all('li')
for i in content_1 :
    print(i.get_text().replace("\n",""))

#Step 8. 총 검색 건수를 보여주고 수집할 건수 입력받기
import math
total_cnt = soup_1.find('div','searchBox').find('span','num').get_text()
print('검색하신 키워드 %s (으)로 총 %s 건의 학위논문이 검색되었습니다' %(query_txt,total_cnt))
collect_cnt = int(input('이 중에서 몇 건을 수집하시겠습니까?: '))
collect_page_cnt = math.ceil(collect_cnt / 10)
print('%s 건의 데이터를 수집하기 위해 %s 페이지의 게시물을 조회합니다.' %(collect_cnt,collect_page_cnt))

#Step 9. 각 항목별로 데이터를 추출하여 리스트에 저장하기
no2 = [ ]        #번호 저장
title2 = [ ]     #논문제목 저장
writer2 = [ ]    #논문저자 저장
org2 = [ ]       #소속기관 저장
year2 = []       #발표년도 저장
paper_zip2 = []  #논문집/발표집 저장
contents2 = []   # 초록 내용          
paper_url2 = []  #논문 url 주소 저장
no = 1

# 다음 페이지 번호 만들기
page_no=[ ]

for i in range(10,collect_cnt) :
    if i % 10 == 0 :
            page_no.append(i + 1)

for a in range(1, collect_page_cnt + 1) :
    
    html_2 = driver.page_source
    soup_2 = BeautifulSoup(html_2, 'html.parser')

    content_2 = soup_2.find('div','srchResultListW').find_all('li')
    
    for b in content_2 :    
        #1. 논문제목 있을 경우만
        try :
            title = b.find('div','cont').find('p','title').get_text()
        except :
            continue
        else :
            print('1.번호:',no)
            no2.append(no)

            print('2.제목:',title)
            title2.append(title)
            
            writer = b.find('span','writer').get_text()
            print('3.작성자:',writer)
            writer2.append(writer)

            org = b.find('span','assigned').get_text()
            print('4.소속기관:' , org)
            org2.append(org)
            
            year = b.find('p','etc').find_all('span')[2].get_text()
            print('5.발표년도:', year)
            year2.append(year)
            
            paper_zip = b.find('p','etc').find_all('span')[3].get_text()
            print('6.논문집/자료집:', paper_zip)
            paper_zip2.append(paper_zip)
            
            paper_url = url + b.find('div','cont').find('p','title').find('a')['href']
            print('7.논문 URL 주소:', paper_url)
            paper_url2.append(paper_url)
                 
            time.sleep(1)
                 
            driver.get(paper_url)
                 
            html_3 = driver.page_source
            soup_1 = BeautifulSoup(html_3, 'html.parser')
            
            try :
                cont=soup_1.find("div","text").find('p').get_text().replace("\n","").strip()
            except :
                cont='초록이 없습니다'
                contents2.append(cont)
                print("8.초록내용 : %s" %cont)
            else :
                contents2.append(cont)
                print("8.초록내용 : %s" %cont)
            
            time.sleep(1)
            
            driver.back() #이전 페이지로 돌아가기
                
            time.sleep(2)
            
            no += 1
            print("\n")
                        
            if no > collect_cnt :
                break

            time.sleep(1)        # 페이지 변경 전 1초 대기 

    c = math.floor(a/10)-1 
    a += 1

    if a == page_no[c]:
        driver.find_element_by_link_text('다음 페이지로').click()
    else :
        driver.find_element_by_link_text('%s' %a).click() # 다음 페이지번호 클릭
        
print("요청하신 작업이 모두 완료되었습니다")

import time
import os
n = time.localtime()
s = '%04d-%02d-%02d-%02d-%02d-%02d' %(n.tm_year, n.tm_mon, n.tm_mday, n.tm_hour, n.tm_min, n.tm_sec)

os.makedirs(dir_path+'RISS'+'-'+s+'-'+'학위논문')
                 
fc_name = dir_path+'RISS'+'-'+s+'-'+'학위논문'+'\\'+'RISS'+'-'+s+'-'+'학위논문'+'.csv'
fx_name = dir_path+'RISS'+'-'+s+'-'+'학위논문'+'\\'+'RISS'+'-'+s+'-'+'학위논문'+'.xls'

# Step 10. 수집된 데이터를 xls와 csv 형태로 저장하기
import pandas as pd 

df = pd.DataFrame()
df['번호']=no2
df['제목']=pd.Series(title2)
df['저자']=pd.Series(writer2)
df['소속(발행)기관']=pd.Series(org2)
df['날짜']=pd.Series(year2)
df['학위(논문일경우)']=pd.Series(paper_zip2)
df['자료URL주소']=pd.Series(paper_url2)
df['초록']=pd.Series(contents2)

# xls 형태로 저장하기
df.to_excel(fx_name,index=False, encoding="utf-8")

# csv 형태로 저장하기
df.to_csv(fc_name,index=False, encoding="utf-8-sig")

print('요청하신 데이터 수집 작업이 정상적으로 완료되었습니다')


 이 크롤러는 RISS 사이트의 논문 및 학술자료 수집용 웹크롤러입니다.
1.수집할 자료의 키워드는 무엇입니까?(여러개일 경우 , 로 구분하여 입력): 해양자원
2. 위 키워드로 아래의 장르 중 어떤 장르의 정보를 수집할까요?
1.학위논문	 2.국내학술논문	 3.해외학술논문	 4.학술지
5.단행본	 6.공개강의	 7.연구보고서
위 장르 중 수집할 장르의 번호를 입력하세요 : 2
3.파일을 저장할 폴더명만 쓰세요(기본값 : e:\BigAI\) : 
국내학술논문
1발표논문 / 해양생물자원으로서 해조류 : 생물활성물질의 정제와 분자적 응용홍용기(Yong Ki Hong)한국조류학회2000국제심포지움 일정 및 발표논문집 - 21세기, 해양환경과 해양생물자원의 전망Vol.- No.-해조생물공학은 유용해조류 및 그들의 유용물질들을 경제적으로 생산할 수 있도록 개발하는 학문 분야이다. 본 논문에서는 이들 해조생물공학에 대한 실험방법적 접근을 강조하여 설명하고자 한다. 우선 해조류의 활력측정은 조직 자체의 효소활성 즉 무색의 2,3,5-triphenyltetrazolium chloride를 붉은색의 triphenylformazan으로 환원시키는 정도를 측정함으로서 정량적으로 간단하게 활력을 평가하는 방법을 확립하였다. 그리고 callus와 엽체 재생 등을 위한 무균 고체 조직배양에 가장 적합한 gelling agent들을 비교한바 high gel strength agar가 톳의 callus 형성 및 엽체 재생에 지지체로 가장 효율적이였다. 유용 생리활성물질의 탐색작업에서는 보라색우무 추출물에서 Taq DNA polymerase에 대한 저해작용을 보였으며, 청각, 구멍갈파래, 잎파래, 미역에는 Herpes simplex virus에 대한 항virus효과를 지녔다. 홑파래물추출물에서는 생먹이용 미세조류들의 성장에 촉진효과를 보였으며, 작은구슬산호말은 적조들의 성장을 억제시켰다. 넓패는 가시파래 및 담치에 대한 부착 억제작용을 보였다. 또한 김과 홑파래는 고cholesterol혈증

이 중에서 몇 건을 수집하시겠습니까?: 11
11 건의 데이터를 수집하기 위해 2 페이지의 게시물을 조회합니다.
1.번호: 1
2.제목: 발표논문 / 해양생물자원으로서 해조류 : 생물활성물질의 정제와 분자적 응용
3.작성자: 홍용기(Yong Ki Hong)
4.소속기관: 한국조류학회
5.발표년도: 2000
6.논문집/자료집: 국제심포지움 일정 및 발표논문집 - 21세기, 해양환경과 해양생물자원의 전망
7.논문 URL 주소: http://www.riss.kr//search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=508c47ca1906d5ecffe0bdc3ef48d419
8.초록내용 : 해조생물공학은 유용해조류 및 그들의 유용물질들을 경제적으로 생산할 수 있도록 개발하는 학문 분야이다. 본 논문에서는 이들 해조생물공학에 대한 실험방법적 접근을 강조하여 설명하...


1.번호: 2
2.제목: 수거된 해양폐기물 자원화 기술 개발(Ⅰ)
3.작성자: 길상인(Sang-In Keel), 윤진한(Jin-Han Yun), 최연석(Yeon-Seok Choi), 강창구(Chang-Gu Kang), 유정석(Jeong-Seok Yu)
4.소속기관: 한국해양환경·에너지학회
5.발표년도: 2002
6.논문집/자료집: 한국해양환경·에너지학회지
7.논문 URL 주소: http://www.riss.kr//search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=09ec1e3541fcc118ffe0bdc3ef48d419
8.초록내용 : 본 연구는 수거된 해양폐기물을 원료로 이용하여 폐기물연료(RDF : Refuse Derived Fuel)를 제조하는 공정 개발을 목적으로 하고 있으며, 산업용 연료로서의 활용 가능성을 확인하기 제조된 RDF의 ...


1.번호: 3
2.제목: 해양경찰 미래 발전을 위한 인적 자원 확보 방안연구
3.작성자: 윤병두
4.소속기관: 한국

2. 청와대 국민 청원 게시판에 있는 다양한 청원들의 '전체 목록'에서 각 청원에 대한 상세 내역을 수집하고 웹 크롤러를 만들세요. 수집된 데이터 형식은 다음 페이지에 있는 각 형식별 수집 예시를 참고하세요. 수집된 데이터 형식은 다음 페이지에 있는 각 형식별 수집 예시를 참고하세요.

[청와대 국민청원 홈페이지 URL] : https://www.president.go.kr/petitions

위와 같은 코롤링할 건수와 파일을 저장할 폴더 정보를 입력 받은 후 전체 목록에서 아래의 수집 할 내용을 수집하도록 코드를 작성하세요.

[수집할 내용]
- 청원번호 / 청원제목 / 참여인원 / 카테고리 / 청원시작일 / 청원종료일 / 청원내용


In [8]:
from selenium import webdriver
import time 
import math

#Step 2. 사용자에게 검색 관련 정보들을 입력 받습니다.
print("=" *100)
print("연습문제 6-2 : 청와대 국민청원 게시판 크롤링하기")
print("=" *100)

#Step 3. 수집된 데이터를 저장할 파일 이름 입력받기 
cont_num = int(input("1.크롤링 할 건수는 몇건입니까? : "))
collect_page_cnt = math.ceil(cont_num / 7)
dir_path = input("2.파일을 저장할 폴더명만 쓰세요(기본값 : e:\\BigAI\\) : ")

if dir_path == '' :
    dir_path = 'e:\\BigAI\\'

#Step 4. 크롬 드라이버 설정 및 웹 페이지 열기
chrome_path = "C:/Users/aa/Downloads/chromedriver.exe"
driver = webdriver.Chrome(chrome_path)

url = 'http://www1.president.go.kr/petitions'
driver.get(url)
time.sleep(5)

from bs4 import BeautifulSoup
html_1 = driver.page_source
soup_1 = BeautifulSoup(html_1, 'html.parser')

content_1 = soup_1.find('div','ct_list1').find_all('li')

for i in content_1 :
    print(i.get_text().replace("\n",""))
    
num2 = []          # 청원번호
title2 = []        # 청원제목
peo_cnt2 = []      # 참여인원
cate2 = []         # 카테고리
start_date2 = []   # 청원시작일
end_date2 = []     # 청원종료일
cont2 = []         # 청원내용
    
for a in range(1, collect_page_cnt + 1) :
    
    html_2 = driver.page_source
    soup_2 = BeautifulSoup(html_2, 'html.parser')

    content_2 = soup_2.find('div','ct_list1').find_all('li')
    
    for b in content_2 :    
        
        num = b.find('div','bl_no').get_text()
        num2.append(num)
        print('1.청원번호 : ', num)        
        
        # 청원별 url 가져오기
        content_url = b.find('div','bl_subject').find('a')['href']
        full_url = url + content_url
        
        driver.get(full_url)
        
        time.sleep(5)
        
        html_3 = driver.page_source
        soup_3 = BeautifulSoup(html_3, 'html.parser')
        
        title = soup_3.find('div', 'petitionsView_left').find('h3', 'petitionsView_title').get_text()
        title2.append(title)
        print('2.청원제목 : ', title)
        
        peo_cnt = soup_3.find('div', 'petitionsView_left_pg').find('h2', 'petitionsView_count').find('span','counter').get_text()
        peo_cnt2.append(peo_cnt)
        print('3.참여인원 : ', peo_cnt)
        
        cate = soup_3.find('div', 'petitionsView_left_pg').find('ul','petitionsView_info_list').find_all('li')[0].get_text()
        cate2.append(cate)
        print('4.카테고리 : ', cate)
        
        start_date = soup_3.find('div', 'petitionsView_left_pg').find('ul','petitionsView_info_list').find_all('li')[1].get_text()
        start_date2.append(start_date)
        print('4.청원시작일 : ', start_date)
        
        end_date = soup_3.find('div', 'petitionsView_left_pg').find('ul','petitionsView_info_list').find_all('li')[2].get_text()
        end_date2.append(end_date)
        print('4.청원종료일 : ', end_date)
        
        cont = soup_3.find('div', 'petitionsView_left_pg').find('div','View_write').get_text()
        cont2.append(cont)
        print('4.청원내용 : ', cont)
        
        driver.back() #이전 페이지로 돌아가기
                
        time.sleep(2)
        

    c = math.floor(a/10)-1 
    a += 1

    if a == page_no[c]:
        driver.find_element_by_link_text('다음 페이지로').click()
    else :
        driver.find_element_by_link_text('%s' %a).click() # 다음 페이지번호 클릭
        
print("요청하신 작업이 모두 완료되었습니다")

import time
import os
n = time.localtime()
s = '%04d-%02d-%02d-%02d-%02d-%02d' %(n.tm_year, n.tm_mon, n.tm_mday, n.tm_hour, n.tm_min, n.tm_sec)

os.makedirs(dir_path+'국민청원'+'-'+s+'-'+'전체청원')
                 
fc_name = dir_path+'국민청원'+'-'+s+'-'+'전체청원'+'\\'+'RISS'+'-'+s+'-'+'학위논문'+'.csv'
fx_name = dir_path+'국민청원'+'-'+s+'-'+'전체청원'+'\\'+'RISS'+'-'+s+'-'+'학위논문'+'.xls'

# Step 10. 수집된 데이터를 xls와 csv 형태로 저장하기
import pandas as pd 

df = pd.DataFrame()
df['청원번호']=pd.Series(num2)
df['청원제목']=pd.Series(title2)
df['참여인원']=pd.Series(peo_cnt2)
df['카테고리']=pd.Series(cate2)
df['청원시작일']=pd.Series(start_date2)
df['청원종료일']=pd.Series(end_date2)
df['청원내용']=pd.Series(cont2)

# xls 형태로 저장하기
df.to_excel(fx_name,index=False, encoding="utf-8")

# csv 형태로 저장하기
df.to_csv(fc_name,index=False, encoding="utf-8-sig")

print('요청하신 데이터 수집 작업이 정상적으로 완료되었습니다')

연습문제 6-2 : 청와대 국민청원 게시판 크롤링하기
1.크롤링 할 건수는 몇건입니까? : 10
2.파일을 저장할 폴더명만 쓰세요(기본값 : e:\BigAI\) : 
번호 734분류 저출산/고령화대책제목 전국의 아이를 원하는 난임부부들에게 힘을주세요.청원 종료일 2021-08-11참여인원 593명
번호 733분류 인권/성평등제목 청소년 노동인권 보호를 위한 청소년 근로 교육을 의무화 해주십시오.청원 종료일 2021-08-11참여인원 172명
번호 732분류 안전/환경제목 전국에 종이팩 수거함을 설치하여 재활용될 수 있도록 지침을 마련해주세요청원 종료일 2021-08-11참여인원 367명
번호 731분류 보건복지제목 타액자가진단키트를 사용하여 아이들의 코로나 감염을 막을 수 있게 해주세요.청원 종료일 2021-08-11참여인원 950명
번호 730분류 농산어촌제목 60년간 농사지은땅 하루아침 농어촌공사에서 공매로 땅장사 하려고 합니다.청원 종료일 2021-08-11참여인원 247명
번호 729분류 보건복지제목 코로나 AZ 1차 백신 맞고 6일 만에 갑자기 의식 잃으신 56년생 아버지, 이례적인 뇌경색 소견으로 돌아가셨습니다.청원 종료일 2021-08-11참여인원 1,039명
번호 728분류 보건복지제목 정치인, 기자, 연예인, **노총 조합원들에게 코로나 백신 우선 접종 청원합니다청원 종료일 2021-08-11참여인원 138명
1.청원번호 :  번호 734


AttributeError: 'NoneType' object has no attribute 'find'

3. 네이버 View -> 블로그 - 검색어와 날자와 검색 건수를 입력 받은 후 네이버 블로그의 상세 내용을 수집하도록 크롤러를 만들세요.

아래 그림과 같이 크롤링할 키워드 / 반드시 포함할 키워드 / 제외할 키워드 / 시작일자 / 종료일자 / 크롤링할 건 수 / 저장할 폴더명의 입력 받은 후 해당 정보로 여러 건의 네이버 블로그를 크롤링하여 txt, csv, xls 형식의 파일로 저장하는 코롤러를 만드세요.